## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

def f_compute_chisqr(dict_val,dict_sample,img_size):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=int(img_size/2)
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_sdev'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_sdev'][:idx]-dict_sample['spec_sdev'][:idx])**2))
        print(spec_loss)
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict

In [6]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
# s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
num_bkgnd=8000
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)
img_size=s_val.shape[1]
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(8000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [14]:
dict_val.keys()

dict_keys(['hist_val', 'hist_err', 'hist_bin_centers', 'spec_val', 'spec_sdev'])

In [15]:
df_runs=pd.DataFrame([])

In [16]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:500][:,:,:,0])
dict1['num_imgs']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


19.03730390968856


In [17]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
prefix='keras_'

for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key=prefix+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:500]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_imgs']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


23.72642724495634
21.61369575112093
25.82132044478256
20.2193980380533


Load LBANN images

In [18]:
parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201112_143907_bsize256_test_with_invtransform-scale4.0'
lst=[(10, 9277), (11, 10281), (16, 14153), (18, 16349)]

prefix='lbann_train_1_'
for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=parent_dir+'/dump_outs/trainer0/model0/'+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_imgs']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)

{'label': 'lbann_train_1_10-9277', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201112_143907_bsize256_test_with_invtransform-scale4.0/dump_outs/trainer0/model0/sgd.training.epoch.10.step.9277_gen_img_instance1_activation_output0.npy', 'num_imgs': 256}
24.980068536144
{'label': 'lbann_train_1_11-10281', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201112_143907_bsize256_test_with_invtransform-scale4.0/dump_outs/trainer0/model0/sgd.training.epoch.11.step.10281_gen_img_instance1_activation_output0.npy', 'num_imgs': 256}
26.73816930285235
{'label': 'lbann_train_1_16-14153', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201112_143907_bsize256_test_with_invtransform-scale4.0/dump_outs/trainer0/model0/sgd.training.epoch.16.step.14153_gen_img_instance1_activation_output0.npy', 'num_imgs': 256}
26.658505171221087
{'label': 'lbann_train_1_18-16349', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128

In [19]:
parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5/'
lst=[(3, 3430), (7, 6460), (8, 7215), (10, 9610), (13, 11725), (16, 14635), (10, 9475)]

prefix='lbann_train_2_'
for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=parent_dir+'/dump_outs/trainer0/model0/'+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_imgs']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)
    


{'label': 'lbann_train_2_3-3430', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5//dump_outs/trainer0/model0/sgd.training.epoch.3.step.3430_gen_img_instance1_activation_output0.npy', 'num_imgs': 256}
29.078899226482584
{'label': 'lbann_train_2_7-6460', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5//dump_outs/trainer0/model0/sgd.training.epoch.7.step.6460_gen_img_instance1_activation_output0.npy', 'num_imgs': 256}
31.08560754029193
{'label': 'lbann_train_2_8-7215', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5//dump_outs/trainer0/model0/sgd.training.epoch.8.step.7215_gen_img_instance1_activation_output0.npy', 'num_imgs': 256}
26.660546816790188
{'label': 'lbann_train_2_10-9610', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5//dump_outs/trainer0/model0/sgd.tr

In [20]:
## LBANN generated images

parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5/gen_imgs_chkpt'
fldr='/epoch.10.step.9475_20201203_075631/dump_outs/trainer0/model0/'
epochs=9

prefix='lbann_gen_1_'
for count,epoch in enumerate(range(epochs)):
    key=prefix+'{0}'.format(epoch)
    fname=parent_dir+fldr+'sgd.testing.epoch.{0}.step.{0}_gen_img_instance1_activation_output0.npy'.format(epoch)
    
    dict1={'label':key,'fname':fname}
    images=np.load(fname)[:,0,:,:]
    dict1['num_imgs']=images.shape[0]
    print(dict1)
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
#     del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)
    


{'label': 'lbann_gen_1_0', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5/gen_imgs_chkpt/epoch.10.step.9475_20201203_075631/dump_outs/trainer0/model0/sgd.testing.epoch.0.step.0_gen_img_instance1_activation_output0.npy', 'num_imgs': 1000}
32.27582240092475
{'label': 'lbann_gen_1_1', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5/gen_imgs_chkpt/epoch.10.step.9475_20201203_075631/dump_outs/trainer0/model0/sgd.testing.epoch.1.step.1_gen_img_instance1_activation_output0.npy', 'num_imgs': 1000}
32.439866741049656
{'label': 'lbann_gen_1_2', 'fname': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5/gen_imgs_chkpt/epoch.10.step.9475_20201203_075631/dump_outs/trainer0/model0/sgd.testing.epoch.2.step.2_gen_img_instance1_activation_output0.npy', 'num_imgs': 1000}
32.4288357050747
{'label': 'lbann_gen_1_3', 'fname': '/global/cscratch1/sd/

In [65]:
# ## LBANN generated images

# parent_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201130_084039_bsize256_test_with_invtransform-scale0.5/gen_imgs_chkpt'
# fldr='/epoch.12.step.10770_20201201_121847/dump_outs/trainer0/model0/'
# epochs=9

# prefix='lbann_gen_2_'
# for count,epoch in enumerate(range(epochs)):
#     key=prefix+'{0}'.format(epoch)
#     fname=parent_dir+fldr+'sgd.testing.epoch.{0}.step.{0}_gen_img_instance1_activation_output0.npy'.format(epoch)
    
#     dict1={'label':key,'fname':fname}
#     images=np.load(fname)[:,0,:,:]
#     dict1['num_imgs']=images.shape[0]
#     print(dict1)
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample,img_size)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
# #     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)
    



In [21]:
# df_runs.columns

In [22]:
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_sdev', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2','chi_spec3', 'num_imgs']
df_runs=df_runs.reindex(columns=col_list)


In [23]:
df_runs

,label,fname,hist_bin_centers,hist_err,hist_val,spec_sdev,spec_val,chi_1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec1,chi_spec2,chi_spec3,num_imgs
0,raw,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0028445197484184732, 0.0020816260811514454,...","[1.121347563774932, 0.9576931243390049, 0.2568...","[44450.167879114895, 22573.11075528218, 15349....","[87592.048827034, 62024.131669237926, 44898.48...",0.008450,0.000121,0.000127,0.008202,0.000028,4.089795,0.001939,0.128687,19.037304,500.0
1,keras_0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003081765924486072, 0.0023479649643570986, ...","[1.1146491994998429, 0.9643402305198981, 0.260...","[42790.708515013386, 21535.157649232824, 14415...","[85906.63125591779, 60940.73459686716, 43971.7...",0.001476,0.000066,0.000471,0.000939,0.000039,4.081442,0.054668,5.546397,23.726427,500.0
2,keras_1,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0031456442853499706, 0.002349065715521853, ...","[1.1148820753244182, 0.9669777732485703, 0.261...","[44993.59584742486, 22703.883553150186, 16263....","[86439.30616832257, 60426.54691713405, 45079.8...",0.001721,0.000433,0.000196,0.001093,0.000091,4.180616,0.070094,6.117566,21.613696,500.0
3,keras_2,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0032299157726108983, 0.00241667307354839, 0...","[1.1439634352627102, 0.9270007224057393, 0.253...","[41984.635273342275, 22377.328892028356, 14242...","[82519.15650125599, 59058.62442671653, 43854.1...",0.003428,0.002287,0.000471,0.000670,0.001813,4.139944,0.020309,1.257560,25.821320,500.0
4,keras_3,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0029680794378720322, 0.002208163391797882, ...","[1.0973298711470636, 0.9833819747814072, 0.269...","[45458.16732584579, 21884.70881964065, 14530.5...","[88968.60526771784, 62091.84879396162, 45370.3...",0.003392,0.001494,0.000660,0.001238,0.001118,4.010530,0.050543,5.391138,20.219398,500.0
5,lbann_train_1_10-9277,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0023391063934671105, 0.0022335526909727592,...","[1.25470130469183, 0.7744354841121593, 0.22908...","[47337.26067446963, 20030.827248683043, 13772....","[89334.27045736834, 61969.07923576086, 47681.8...",0.082320,0.057603,0.015766,0.008952,0.054249,3.932783,0.184342,24.843320,24.980069,256.0
6,lbann_train_1_11-10281,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002140105934911457, 0.002300315285323828, 0...","[1.0343721833683248, 1.041833685453226, 0.3013...","[43563.208030971204, 18299.18665332431, 10847....","[91100.46101730876, 61555.836609803926, 44107....",0.056598,0.028353,0.017647,0.010599,0.016332,3.788437,0.178319,26.025031,26.738169,256.0
7,lbann_train_1_16-14153,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0021792015981071376, 0.002284176979785651, ...","[1.089828330109229, 1.0108991653397892, 0.2556...","[46238.97854802502, 21255.69335365002, 12831.8...","[90000.46403981373, 59835.78898313529, 43880.3...",0.011836,0.004490,0.003530,0.003817,0.003535,4.280449,0.545406,62.201898,26.658505,256.0
8,lbann_train_1_18-16349,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.002622181085149457, 0.0030105302569368034, ...","[1.1674780130697535, 0.9232765599406292, 0.225...","[43535.42943180019, 16197.438276177973, 12158....","[80202.06370343873, 53433.11254794436, 40775.3...",0.018249,0.013350,0.001280,0.003619,0.005439,4.388958,1.001686,104.347189,28.790125,256.0
9,lbann_train_2_3-3430,/global/cscratch1/sd/vpa/proj/cosmogan/results...,"[-1.031746031584782, -0.6161616169043975, -0

### Compare chi-squares

In [24]:
chi_keys=['label','chi_1','chi_spec1', 'chi_spec3', 'num_imgs', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','fname']
df_runs[chi_keys]

,label,chi_1,chi_spec1,chi_spec3,num_imgs,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,fname
0,raw,0.008450,0.001939,19.037304,500.0,0.000121,0.000127,0.008202,0.000028,4.089795,0.128687,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_0,0.001476,0.054668,23.726427,500.0,0.000066,0.000471,0.000939,0.000039,4.081442,5.546397,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_1,0.001721,0.070094,21.613696,500.0,0.000433,0.000196,0.001093,0.000091,4.180616,6.117566,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,keras_2,0.003428,0.020309,25.821320,500.0,0.002287,0.000471,0.000670,0.001813,4.139944,1.257560,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
4,keras_3,0.003392,0.050543,20.219398,500.0,0.001494,0.000660,0.001238,0.001118,4.010530,5.391138,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,lbann_train_1_10-9277,0.082320,0.184342,24.980069,256.0,0.057603,0.015766,0.008952,0.054249,3.932783,24.843320,/global/cscratch1/sd/vpa/proj/cosmogan/results...
6,lbann_train_1_11-10281,0.056598,0.178319,26.738169,256.0,0.028353,0.017647,0.010599,0.016332,3.788437,26.025031,/global/cscratch1/sd/vpa/proj/cosmogan/results...
7,lbann_train_1_16-14153,0.011836,0.545406,26.658505,256.0,0.004490,0.003530,0.003817,0.003535,4.280449,62.201898,/global/cscratch1/sd/vpa/proj/cosmogan/results...
8,lbann_train_1_18-16349,0.018249,1.001686,28.790125,256.0,0.013350,0.001280,0.003619,0.005439,4.388958,104.347189,/global/cscratch1/sd/vpa/proj/cosmogan/results...
9,lbann_train_2_3-3430,0.051595,2.708747,29.078899,256.0,0.046158,0.000864,0.004573,0.039143,5.302889,228.700179,/global/cscratch1/sd/vpa/proj/cosmogan/results...


In [25]:
dict_val['spec_val']
dict_val.keys()


dict_keys(['hist_val', 'hist_err', 'hist_bin_centers', 'spec_val', 'spec_sdev'])

In [31]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')



In [33]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('raw', 'keras_0', 'keras_1', 'keras_2…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [89]:
col_list=['label','chi_1','chi_spec1','num_imgs','chi_2','chi_spec2']
df_runs[col_list]

,label,chi_1,chi_spec1,num_img,chi_2,chi_spec2
0,raw,0.008450,0.001939,500.0,0.000028,0.128687
1,keras_0,0.001476,0.054668,500.0,0.000039,5.546397
2,keras_1,0.001721,0.070094,500.0,0.000091,6.117566
3,keras_2,0.003428,0.020309,500.0,0.001813,1.257560
4,keras_3,0.003392,0.050543,500.0,0.001118,5.391138
5,lbann_train_1_10-9277,0.082320,0.184342,256.0,0.054249,24.843320
6,lbann_train_1_11-10281,0.056598,0.178319,256.0,0.016332,26.025031
7,lbann_train_1_16-14153,0.011836,0.545406,256.0,0.003535,62.201898
8,lbann_train_1_18-16349,0.018249,1.001686,256.0,0.005439,104.347189
9,lbann_train_2_16-14211,0.012966,0.599636,256.0,0.001626,81.202138


In [88]:
# f_plot_intensity_grid(images[:20],cols=5)
# f_pixel_intensity(f_transform(images),bins=100)

